<a href="https://colab.research.google.com/github/ObliviousAI/enclave_workshops/blob/master/enclave_workshops/workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/Workshop_2_Synthetic_Data_from_Multiple_Private_Data_Sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop 2: Synthetic Data from Multiple Private Data Sources

This can all be done form your local machine, but for easy reproducibility we'll user a Google Colab.

### Step 1: Install the oblv proxy

In [ ]:
%%bash
# downlaod the OBLV CLI package for Linux 
wget https://oblv-cli-binary.s3.us-east-2.amazonaws.com/0.3.0/oblv-ccli-0.3.0-x86_64-unknown-linux-musl.zip
# unzip it
unzip oblv-ccli-0.3.0-x86_64-unknown-linux-musl.zip
# cd to the folder where binary is present
cd oblv-ccli-0.3.0-x86_64-unknown-linux-musl/
# update execute permissions for the binary
chmod +x $(pwd)/oblv
# create symlink
sudo ln -s $(pwd)/oblv /usr/local/bin/oblv

### Step 2.a: Create Keys for Alice and Bob (Skip if using example keys provided in the workshop)

In [ ]:
!oblv keygen --key-name alice --output /content/

In [ ]:
!oblv keygen --key-name bob --output /content/

### Step 2.b: Load in the public/private example keys from GitHub

In [ ]:
import base64
import requests

files = [
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/alice_private.cer",
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/alice_public.cer",
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/bob_private.cer",
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/bob_public.cer"
]

for file_name in files:
  url = 'https://raw.githubusercontent.com/ObliviousAI/enclave_workshops/master/'+file_name
  req = requests.get(url)
  if req.status_code == requests.codes.ok:
      file_to_save = file_name.split('/')[-1]

      with open('/content/' + file_to_save, 'wb') as file:
        file.write(req.content)
  else:
      print('Content was not found.')

### Step 3: Create a secure enclave connection from the localhost (in Google Colab) to the desired enclave

In [ ]:
%%script bash --bg

oblv connect \
--pcr0 29f5474040db784ec31cbc9f3a77ba54fbaf00743bda9d228b703f28885095707e66f02a3655860d6362ee4fd66672a4 \
--pcr1 bcdf05fefccaa8e55bf2c8d6dee9e79bbff31e34bf28a99aa19e6b29c37ee80b214a414b7607236edf26fcb78654e63f \
--pcr2 fcdd286368b8e2978a004b642782f740094456ffc2a1742ac299a404576f7c655959f7e5f4f436ba9fbb2711d93bc4d8 \
--private-key "/content/alice_private.der" \
--public-key "/content/alice_public.der" \
--url https://oblv-appli-1d4gv4nsdw743-1233300661.enclave.oblivious.ai/ \
--port 443 --lport 3030 

In [ ]:
%%script bash --bg

oblv connect \
--pcr0 29f5474040db784ec31cbc9f3a77ba54fbaf00743bda9d228b703f28885095707e66f02a3655860d6362ee4fd66672a4 \
--pcr1 bcdf05fefccaa8e55bf2c8d6dee9e79bbff31e34bf28a99aa19e6b29c37ee80b214a414b7607236edf26fcb78654e63f \
--pcr2 fcdd286368b8e2978a004b642782f740094456ffc2a1742ac299a404576f7c655959f7e5f4f436ba9fbb2711d93bc4d8 \
--private-key "/content/bob_private.der" \
--public-key "/content/bob_public.der" \
--url https://oblv-appli-1d4gv4nsdw743-1233300661.enclave.oblivious.ai/ \
--port 443 --lport 3031 

### Step 4: Interact with the enclave by sending traffic to the localhost on the port specified

In [ ]:
import base64
import requests

files = [
    "tests/example_data/alice.csv",
    "tests/example_data/bob.csv"
]

for file_name in files:
  url = 'https://raw.githubusercontent.com/oblivious-demo/oblv-smartnoise/master/'+file_name
  req = requests.get(url)
  if req.status_code == requests.codes.ok:
      file_to_save = file_name.split('/')[-1]

      with open('/content/' + file_to_save, 'wb') as file:
        file.write(req.content)
  else:
      print('Content was not found.')

In [ ]:
import requests
import json

ALICE_PROXY = "http://localhost:3030"
BOB_PROXY = "http://localhost:3031"

In [ ]:
r = requests.get(ALICE_PROXY+"/details")

print(json.loads(r.content))

In [ ]:
r = requests.post(ALICE_PROXY+"/upload_data", files={"file": open("/content/alice.csv", 'rb')})

print(json.loads(r.content))

In [ ]:
r = requests.post(BOB_PROXY+"/upload_data", files={"file": open("/content/bob.csv", 'rb')})

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/confirm_data")

print(json.loads(r.content))

In [ ]:
r = requests.get(BOB_PROXY+"/confirm_data")

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/formats")

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/columns", params={"party": "mine"})

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/columns", params={"party": "theirs"})

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/samples", params={"format": "csv", "num_samples":100})

result_file = "/content/output.csv"

open(result_file, 'wb').write(r.content)


In [ ]:
import pandas as pd

df = pd.read_csv("/content/output.csv")

df.head()